# Data Analysis-- Run in silico screening and get predictions
* Created on 02 05 2023 by Jie Shi
* this code is used for prediction of the capacity fold change from all possible combinations of 25 genes
* __ensemble: average from the 10 CNN+LSTM models__

In [1]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import time
from tensorflow import keras

In [2]:
import pickle # for saving ML models

In [3]:
from itertools import product # for making comprehensive (0,1) list
from itertools import combinations # for making comprehensive (0,1) list

In [4]:
data_head=['Pex1', 'Pex2', 'Pex3', 'Pex4', 'Pex5', 'Pex6', 'Pex8', 'Pex10', 
           'Pex11', 'Pex12', 'Pex13', 'Pex14', 'Pex15', 'Pex17', 'Pex19', 'Pex22', 
           'Pex28', 'Pex29', 'Pex30', 'Pex31', 'Pex32', 'Pex34', 'Sei1', 'Rtn1', 
           'Vps1', 'Capacity change (Fold increase compared to WT control)']

In [5]:
gene_num = 25

In [6]:
## create all possible combinations of genes
screen_data = pd.DataFrame( list(product([0, 1], repeat=gene_num)),columns = data_head[:-1])

## CNN+LSTM

In [7]:
screen_data.shape

(33554432, 25)

In [8]:
num_classes=1

#screen_data = final_screen.drop(data_head[-1],axis=1)

## reshape into (n, gene_num, 1) for deep learning model
#screen_data_channel = screen_data.iloc[:,:-1].to_numpy().reshape(screen_data.shape[0],gene_num,1)

print(screen_data.shape)


(33554432, 25)


In [ ]:
## cnn+lst predictions

for i in range(10):
    start = time.time()
    print(i)
    y_pred_screen = pd.DataFrame()

# load the saved modoel
    model_i = keras.models.load_model(f'save_ML/CNN+LSTM_model_05152023_tuning_{i+1}.h5')
    y_pred_screen[str(i)] = np.expm1(model_i.predict(screen_data).squeeze())
    end = time.time()
    print(f"calculate take: {(start-end)/60} minutes")
    # save individual predictions for later usage
    y_pred_screen.to_csv(f"data_screen/screen_prediction_05152023_nodup_cnn_lstm_{i+1}.csv",index=False)
    #screen_data = final_screen.drop(data_head[-1],axis=1)

In [11]:
# load the saved predictions
y_pred_screen = pd.DataFrame()

for i in range(10):
    y_pred_i = pd.read_csv(f"data_screen/screen_prediction_05152023_nodup_cnn_lstm_{i+1}.csv")
    y_pred_screen[str(i)] = y_pred_i#.drop(columns=y_pred_i.columns[0])
    
print(y_pred_screen.shape)

(33554432, 10)


In [12]:
# calculate ensemble average
y_pred_mean = y_pred_screen.mean(axis=1)
y_pred_mean.shape

(33554432,)

In [13]:
final_screen = screen_data
final_screen[data_head[-1]]=y_pred_mean
final_screen=final_screen.sort_values(data_head[-1], ascending=False)
## save the top 5000
final_screen.head(5000).to_csv("data_screen/screen_prediction_05152023_nodup_cnn_lstm_ensemble.csv")